Scotland-Housing-Stock-by-Tenure Stock by Tenure and Stock by tenure by LA

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path
    from io import BytesIO

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www2.gov.scot/Resource/0054/00540622.xls'
    inputFile = sourceFolder / '00540622.xls'

In [2]:
tab = loadxlstabs(inputFile, sheetids = 'Tbl Stock by Tenure')[0]
cell = tab.filter('Estimated stock of dwellings by tenure: 1993 to 2017')
year = cell.fill(DOWN).is_not_blank().is_not_whitespace()
tenure =  cell.shift(0,2).fill(RIGHT).is_not_blank().is_not_whitespace() | \
            cell.shift(0,3).fill(RIGHT).is_not_blank().is_not_whitespace()
observations = tenure.fill(DOWN).is_not_blank().is_not_whitespace().is_number()

Loading in\00540622.xls which has size 86528 bytes
Table names: ['Tbl Stock by Tenure']


In [3]:
Dimensions = [
            HDim(year,'Year',DIRECTLY,LEFT),
            HDim(tenure,'Tenure',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit', 'dwellings-thousands')
            ]
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
table = c1.topandas()

In [4]:
tidy = pd.DataFrame()

In [5]:
table['Tenure'] = table['Tenure'].map(
    lambda x: {
        'Total number of dwellings (000s)' : 'total', 
        'Owner occupied' : 'privately-owned-dwellings/owner-occupied',
       'Rented privately or with a job/business (note this includes households living rent-free) 3' : 'privately-owned-dwellings/rented-privately-or-with-a-job-business',
       'From housing associations 4' : 'socially-rented-dwellings/from-housing-associations',
       'From local authorities, New Towns, Scottish Homes' : 'socially-rented-dwellings/from-local-authorities-new-towns-scottish-homes',
       'Total number of occupied dwellings (000s)' : 'occupied',
       'Total number of vacant dwellings (000s)' : 'all-vacants',
       'Total number occupied dwellings' : 'privately-owned-dwellings/occupied',
       'Vacant private dwellings and second homes': 'privately-owned-dwellings/vacant-private-dwellings-and-second-homes'
        }.get(x, x))

In [6]:
table['Year'] = table['Year'].str.strip()

In [7]:
table['Period'] = table['Year'].astype(str).str[-4:] + ' ' +  table['Year'].astype(str).str[:3]

In [8]:
import re
YEAR_RE = re.compile(r'[0-9]{4}')
YEAR_MONTH_RE = re.compile(r'([0-9]{4})\s+(JAN|FEB|Mar|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|Dec)')
YEAR_QUARTER_RE = re.compile(r'([0-9]{4})\s+(Q[1-4])')

class Re(object):
  def __init__(self):
    self.last_match = None
  def fullmatch(self,pattern,text):
    self.last_match = re.fullmatch(pattern,text)
    return self.last_match

def time2period(t):
    gre = Re()
    if gre.fullmatch(YEAR_RE, t):
        return f"year/{t}"
    elif gre.fullmatch(YEAR_MONTH_RE, t):
        year, month = gre.last_match.groups()
        month_num = {'JAN': '01', 'FEB': '02', 'Mar': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                     'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'Dec': '12'}.get(month)
        return f"month/{year}-{month_num}"
    elif gre.fullmatch(YEAR_QUARTER_RE, t):
        year, quarter = gre.last_match.groups()
        return f"quarter/{year}-{quarter}"
    else:
        print(f"no match for {t}")

table['Period'] = table['Period'].apply(time2period)

In [9]:
import numpy as np
table['OBS'].replace('', np.nan, inplace=True)
table.dropna(subset=['OBS'], inplace=True)
table.rename(columns={'OBS': 'Value'}, inplace=True)
table['Value'] = table['Value'].astype(int)

In [10]:
table['Geography'] = 'S92000003'

In [11]:
table = table[['Period','Geography','Tenure','Measure Type','Value','Unit']]

In [12]:
tidy = pd.concat([tidy , table])

In [13]:
tab1 = loadxlstabs(inputFile, sheetids = 'Tbl Stock by tenure by LA')[0]
cell1 = tab1.filter('Estimated stock of dwellings by tenure and local authority: 2017')
geo = cell1.fill(DOWN).is_not_blank().is_not_whitespace()
tenure1 =  cell1.shift(0,1).fill(RIGHT).is_not_blank().is_not_whitespace() | \
            cell1.shift(0,2).fill(RIGHT).is_not_blank().is_not_whitespace()
observations1 = tenure1.fill(DOWN).is_not_blank().is_not_whitespace().is_number()

Loading in\00540622.xls which has size 86528 bytes
Table names: ['Tbl Stock by tenure by LA']


In [14]:
Dimensions1 = [
            HDim(geo,'Geography',DIRECTLY,LEFT),
            HDim(tenure1,'Tenure',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit', 'dwellings-thousands')
            ]
c2 = ConversionSegment(observations1, Dimensions1, processTIMEUNIT=True)

In [15]:
table1 = c2.topandas()

In [16]:
table1['Tenure'] = table1['Tenure'].map(
    lambda x: {
        'Total number of dwellings (000s)' : 'total', 
        'Owner occupied' : 'privately-owned-dwellings/owner-occupied',
       'Rented privately or with a job/business (note this includes households living rent-free) 3' : 'privately-owned-dwellings/rented-privately-or-with-a-job-business',
       'From housing associations 4' : 'socially-rented-dwellings/from-housing-associations',
       'From local authorities, New Towns, Scottish Homes' : 'socially-rented-dwellings/from-local-authorities-new-towns-scottish-homes',
       'Total number of occupied dwellings (000s)' : 'occupied',
       'Total number of vacant dwellings (000s)' : 'all-vacants',
       'Total number occupied dwellings' : 'privately-owned-dwellings/occupied',
       'Vacant private dwellings and second homes': 'privately-owned-dwellings/vacant-private-dwellings-and-second-homes'
        }.get(x, x))

In [17]:
table1['Period'] = 'month/2017-03'

In [18]:
import numpy as np
table1['OBS'].replace('', np.nan, inplace=True)
table1.dropna(subset=['OBS'], inplace=True)
table1.rename(columns={'OBS': 'Value'}, inplace=True)
table1['Value'] = table1['Value'].astype(int)

In [19]:
table1['Geography'] = table1['Geography'].map(
    lambda x: {
        'Aberdeen City':'S12000033',
        'Aberdeenshire':'S12000034',
        'Angus':'S12000041',
        'Argyll & Bute':'S12000035',
        'Edinburgh, City of':'S12000036',
        'Clackmannanshire':'S12000005',
        'Dumfries & Galloway':'S12000006',
        'Dundee City':'S12000042',
        'East Ayrshire':'S12000008',
        'East Dunbartonshire':'S12000045',
        'East Lothian':'S12000010',
        'East Renfrewshire':'S12000011',
        'Falkirk':'S12000014',
        'Fife':'S12000015',
        'Glasgow City':'S12000046',
        'Highland':'S12000017',
        'Inverclyde':'S12000018',
        'Midlothian':'S12000019',
        'Moray':'S12000020',
        'Na h-Eileanan Siar':'S12000013',
        'North Ayrshire':'S12000021',
        'North Lanarkshire':'S12000044',
        'Orkney Islands':'S12000023',
        'Perth & Kinross':'S12000024',
        'Renfrewshire':'S12000038',
        'Scottish Borders':'S12000026',
        'Shetland Islands':'S12000027',
        'South Ayrshire':'S12000028',
        'South Lanarkshire':'S12000029',
        'Stirling':'S12000030',
        'West Dunbartonshire':'S12000039',
        'West Lothian':'S12000040',
        'Scotland':'S92000003'        
        }.get(x, x))

In [20]:
table1 = table1[['Period','Geography','Tenure','Measure Type','Value','Unit']]

In [21]:
tidy = pd.concat([tidy , table1])

In [22]:
out = Path('out')
out.mkdir(exist_ok=True)
tidy.to_csv(out / 'observations.csv', index = False)

In [23]:
tidy

,Period,Geography,Tenure,Measure Type,Value,Unit
0,month/1993-12,S92000003,total,Count,2193,dwellings-thousands
1,month/1993-12,S92000003,privately-owned-dwellings/owner-occupied,Count,1217,dwellings-thousands
2,month/1993-12,S92000003,privately-owned-dwellings/rented-privately-or-...,Count,154,dwellings-thousands
3,month/1993-12,S92000003,socially-rented-dwellings/from-housing-associa...,Count,67,dwellings-thousands
4,month/1993-12,S92000003,socially-rented-dwellings/from-local-authoriti...,Count,755,dwellings-thousands
5,month/1994-12,S92000003,total,Count,2210,dwellings-thousands
6,month/1994-12,S92000003,privately-owned-dwellings/owner-occupied,Count,1258,dwellings-thousands
7,month/1994-12,S92000003,privately-owned-dwellings/rented-privately-or-...,Count,155,dwellings-thousands
8,month/1994-12,S92000003,socially-rented-dwellings/from-housing-associa...,Count,77,dwellings-thousands
9,month/1994-12,S92000003,socially-rented-dwellings/from-local-authoriti...,Count,721,dwellings-thousands
